In [1]:
import numpy as np
# Constants
MIN_KERNEL_SIZE = 5
MIN_KERNEL_SIZE_GABOR = 2
QUBITS = 4
QC_REPETITIONS = 1
IMAGE_WIDTH = 32
CLASSES = 4
EPOCHS = 10
BATCH_SIZE = 256
PERTURB = False

## Settings
DEBUG = False
FILENAME_PREFIX = "cnn-fashion-mnist"
CHOOSE_NEW_TRAINING_DATA = True

TARGETS = np.array(["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"])


In [2]:
# Necessary imports

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from itertools import cycle
import random

from datetime import datetime

# Additional torch-related imports
import torch
from torch import Tensor, cat, no_grad, manual_seed
from torch.nn import Linear, CrossEntropyLoss, MSELoss, Softmax
from torch.optim import LBFGS
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim
from torch.nn import (
    Module,
    Conv2d,
    Linear,
    Bilinear,
    Dropout2d,
    NLLLoss,
    MaxPool2d,
    Flatten,
    Sequential,
    ReLU,
    BatchNorm2d,
)
import torch.nn.functional as F

# Project specific imports
from canonicallayers.perturbations import skew, rotate, noise, elastic_transform


In [3]:
# Train Dataset
# -------------

# Set train shuffle seed (for reproducibility)
manual_seed(42)

batch_size = BATCH_SIZE
n_samples = CLASSES * 10000  # We will concentrate on the first 6000 * CLASSES samples

if(CHOOSE_NEW_TRAINING_DATA):
  # Use pre-defined torchvision function to load MNIST train data
  X_train = datasets.FashionMNIST(
      root="./data", train=True, download=True, transform=transforms.Compose([transforms.ToTensor()])
  )

  # Filter out labels (originally 0-9), leaving only labels 0 and 1
  idx = np.where(X_train.targets < CLASSES)[0][:n_samples]
  X_train.data = X_train.data[idx]
  X_train.targets = X_train.targets[idx]

  print("X_train.data", X_train.data.shape)
  # Define torch dataloader with filtered data
  train_loader = DataLoader(X_train, batch_size=batch_size, shuffle=True)
  torch.save(train_loader, f'checkpoints/{FILENAME_PREFIX}-training-{CLASSES}.pt')
else:
  train_loader = torch.load(f'checkpoints/{FILENAME_PREFIX}-training-{CLASSES}.pt')

X_train.data torch.Size([24000, 28, 28])


In [4]:
n_samples_show = 10

data_iter = iter(train_loader)
fig, axes = plt.subplots(nrows=2, ncols=n_samples_show, sharex=True, figsize=(15, 3))
images, targets = data_iter.__next__()
print("images shape", images.shape)

while n_samples_show > 0:
    originalImage = images[n_samples_show,0]
      
    if(PERTURB):
      perturbation = "none"
      match random.randint(0, 3):
          case 0:
            perturbedImage = skew(originalImage.numpy())
            perturbation = "skew"
          case 1:
            angle = 3*np.random.random()/5
            perturbedImage = rotate(originalImage.numpy(), angle)
            perturbation = f"rotate {np.round(angle*180/np.pi)}°"
          case 2:
            perturbedImage = noise(originalImage.numpy())
            perturbation = "noise"
          case _:
            perturbedImage = elastic_transform(originalImage.numpy())
            perturbation = "elastic"
      perturbedData = torch.tensor(perturbedImage).unsqueeze(0).unsqueeze(0)

    axes[0,n_samples_show - 1].imshow(originalImage.numpy().squeeze(), cmap="gray")
    axes[0,n_samples_show - 1].set_xticks([])
    axes[0,n_samples_show - 1].set_yticks([])
    axes[0,n_samples_show - 1].set_title(TARGETS[targets[n_samples_show].item()])
    if(PERTURB):
      axes[1,n_samples_show - 1].imshow(perturbedData[0].numpy().squeeze(), cmap="gray")
      axes[1,n_samples_show - 1].set_xticks([])
      axes[1,n_samples_show - 1].set_yticks([])
      axes[1,n_samples_show - 1].set_title("{}".format(perturbation))

    n_samples_show -= 1

images shape torch.Size([256, 1, 28, 28])


In [5]:
# Test Dataset
# -------------

# Set test shuffle seed (for reproducibility)
manual_seed(5)

n_samples = CLASSES * 1000  # We will concentrate on 1000 * CLASSES samples

if(CHOOSE_NEW_TRAINING_DATA):
  # Use pre-defined torchvision function to load MNIST test data
  X_test = datasets.FashionMNIST(
      root="./data", train=False, download=True, transform=transforms.Compose([transforms.ToTensor()])
  )

  # Filter out labels (originally 0-9), leaving only labels 0 and 1
  idx = np.where(X_test.targets < CLASSES)[0][:n_samples]
  X_test.data = X_test.data[idx]
  X_test.targets = X_test.targets[idx]

  # Define torch dataloader with filtered data
  test_loader = DataLoader(X_test, batch_size=batch_size, shuffle=False)
  torch.save(test_loader, f'checkpoints/{FILENAME_PREFIX}-testing-{CLASSES}.pt')
else:
  test_loader = torch.load(f'checkpoints/{FILENAME_PREFIX}-testing-{CLASSES}.pt')

In [6]:
# Define torch NN module
OUT_CH = 2

class Net(Module):
    def __init__(self):
      super().__init__()
      self.conv1 = Conv2d(1, OUT_CH, kernel_size=4, stride=2, padding=1)
      self.bn1 = BatchNorm2d(OUT_CH)
      self.relu1 = ReLU()
      self.mp1 = MaxPool2d(kernel_size=3, stride=3) # output: 64 x 16 x 16

      self.conv2 = Conv2d(OUT_CH, 2*OUT_CH, kernel_size=3)
      self.bn2 = BatchNorm2d(2*OUT_CH)
      self.relu2 = ReLU()
      self.mp2 = MaxPool2d(2, 2) # output: 128 x 8 x 8

      self.fc1 = Linear(2*OUT_CH*6*6, 600)
      self.drop = Dropout2d(0.25)
      self.fc2 = Linear(600, 120)
      self.fc3 = Linear(120, CLASSES)

    def forward(self, x):
      # print("first", x.shape)
      x = self.conv1(x)
      # print("after conv1", x.shape)
      x = self.bn1(x)
      # print("after bn1", x.shape)
      x = self.relu1(x)
      # print("after relu1", x.shape)
      x = self.mp1(x)
      # print("after mp1", x.shape)

      x = self.conv2(x)
      # print("after conv2", x.shape)
      x = self.bn2(x)
      # print("after bn2", x.shape)
      x = self.relu2(x)
      # print("after relu2", x.shape)
      x = self.mp2(x)
      # print("after mp2", x.shape)
      x = x.view(x.size(0), -1)
      # print("after conv2, bn2, relu2, mp2", x.shape)
      # x = self.fc1(x)
      # x = self.drop(x)
      # x = self.fc2(x)
      # x = self.fc3(x)

      return x


model = Net()

In [7]:
# Define model, optimizer, and loss function
start_time = datetime.now()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_func = CrossEntropyLoss()

# Start training
epochs = EPOCHS  # Set number of epochs
loss_list = []  # Store loss history
model.train()  # Set model to training mode
# m = Softmax(dim=3)

print("Training Set size:", len(train_loader.dataset), "Epochs", EPOCHS, "Batch Size:", BATCH_SIZE, "Number of batches:", (len(train_loader.dataset)//BATCH_SIZE)+1)
total_loss = []
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad(set_to_none=True)  # Initialize gradient
        if(DEBUG):
          print("epoch", epoch, "batch", batch_idx)
        output = model(data)  # Forward pass
        loss = loss_func(output, target)  # Calculate loss
        print(epoch, batch_idx, "Loss", loss.item())
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize weights
        total_loss.append(loss.item())  # Store loss
    loss_list.append(sum(total_loss) / len(total_loss))
    print("Training [{:.0f}%]\tLoss: {:.4f}".format(100.0 * (epoch + 1) / epochs, loss_list[-1]))
    torch.save(model.state_dict(), f'checkpoints/{FILENAME_PREFIX}-q{QUBITS}-r{QC_REPETITIONS}-c{CLASSES}-{epoch}.pt')
end_time = datetime.now()
elapsed = end_time - start_time
print("Training Time: {:.0f} seconds]".format(elapsed.total_seconds()))

Training Set size: 24000 Epochs 10 Batch Size: 256 Number of batches: 94
0 0 Loss 1.6333175897598267
0 1 Loss 1.4864017963409424
0 2 Loss 1.5854040384292603
0 3 Loss 1.5963834524154663
0 4 Loss 1.487963080406189
0 5 Loss 1.475978970527649
0 6 Loss 1.453248143196106
0 7 Loss 1.4391392469406128
0 8 Loss 1.4059951305389404
0 9 Loss 1.5397809743881226
0 10 Loss 1.413959264755249
0 11 Loss 1.445461630821228
0 12 Loss 1.3938442468643188
0 13 Loss 1.448526382446289
0 14 Loss 1.450868010520935
0 15 Loss 1.371293306350708
0 16 Loss 1.4463480710983276
0 17 Loss 1.3315778970718384
0 18 Loss 1.3842612504959106
0 19 Loss 1.3457183837890625
0 20 Loss 1.3270961046218872
0 21 Loss 1.2837209701538086
0 22 Loss 1.3018893003463745
0 23 Loss 1.3636795282363892
0 24 Loss 1.3505346775054932
0 25 Loss 1.3059766292572021
0 26 Loss 1.3219112157821655
0 27 Loss 1.277825117111206
0 28 Loss 1.2670472860336304
0 29 Loss 1.264510154724121
0 30 Loss 1.235926866531372
0 31 Loss 1.22247314453125
0 32 Loss 1.2036134004

In [8]:
# Plot loss convergence
plt.plot(total_loss)
plt.title("CNN - Training Convergence")
plt.xlabel("Training Batches")
plt.ylabel("Cross Entropy Loss")
plt.show()

In [9]:
def plotKernel(ax, locations, kernels, tunings, title):
  ax.set_xticks([])
  ax.set_yticks([])
  ax.set_title(title)
  imgWidth = IMAGE_WIDTH - MIN_KERNEL_SIZE +1
  ax.plot([0, 0],[IMAGE_WIDTH, IMAGE_WIDTH])
  cycol = cycle('bgrcmk')
  Y = []
  X = []
  C=[]
  S=[]
  kernelSize=[]
  tuningX=[]
  tuningY=[]
  tuningValue=[]
  for iy, ix in np.ndindex(locations.shape):
    y = locations[iy, ix] // imgWidth
    x = locations[iy, ix] % imgWidth
    Y.append(y)
    X.append(x)
    ks = kernels[iy, ix].item()
    kernelSize.append(ks * ks * 2000/(IMAGE_WIDTH*IMAGE_WIDTH))
    tuningX.append(x + ks/2)
    tuningY.append(y + ks/2)
    size = kernels[iy, ix].item()
    S.append(size)
    tuningValue.append(tunings[iy, ix].item())
    ax.add_patch(Rectangle((x.item(), y.item()), size, size, facecolor= 'none', edgecolor=next(cycol), alpha=0.75))
  ax.scatter(tuningX, tuningY,c=tuningValue, s=kernelSize, cmap='Greys', alpha=0.75)
    # print(x.item(), y.item(), kernels[iy, ix].item())



In [10]:
n_samples_show = 6
count = 0


with no_grad():
  imgWidth = IMAGE_WIDTH - MIN_KERNEL_SIZE +1

  
  print("Testing Set size:", len(test_loader.dataset), "Batch Size:", BATCH_SIZE, "Number of batches:", (len(test_loader.dataset)//BATCH_SIZE)+1)

  for epoch in range(EPOCHS):
    model = Net()
    model.load_state_dict(torch.load(f'checkpoints/{FILENAME_PREFIX}-q{QUBITS}-r{QC_REPETITIONS}-c{CLASSES}-{epoch}.pt'))
    model.eval()  # set model to evaluation mode
    correct = 0
    total_loss = []

    for batch_idx, (data, target) in enumerate(test_loader):
      perturbations = []
      perturbedData = data
      if(PERTURB):
        perturbedData = torch.zeros_like(perturbedData)
        for bi in range(0, data.shape[0]):
          originalImage = data[bi,0]
          match random.randint(0, 3):
              case 0:
                perturbedImage = skew(originalImage.numpy())
                perturbations.append("skew")
              case 1:
                angle = 3*np.random.random()/5
                perturbedImage = rotate(originalImage.numpy(), angle)
                perturbations.append(f"rotate {np.round(angle*180/np.pi)}°")
              case 2:
                perturbedImage = noise(originalImage.numpy())
                perturbations.append("noise")
              case _:
                perturbedImage = elastic_transform(originalImage.numpy())
                perturbations.append("elastic")
          perturbedData[bi] = torch.tensor(perturbedImage).unsqueeze(0)

      output = model(perturbedData)
      # print("prediction output", output)
      print(batch_idx+1, end=" ")
      if len(output.shape) == 1:
          output = output.reshape(1, *output.shape)

      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()

      loss = loss_func(output, target)
      total_loss.append(loss.item())
      count += 1

    print(
        "Performance on test data:\n\tLoss: {:.4f}\n\tClasses: {}, \n\tAccuracy: {:.1f}%".format(
            sum(total_loss) / len(total_loss), 
            CLASSES, correct / len(test_loader) / batch_size * 100
        )
    )


Testing Set size: 4000 Batch Size: 256 Number of batches: 16
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 Performance on test data:
	Loss: 0.9432
	Classes: 4, 
	Accuracy: 62.8%
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 Performance on test data:
	Loss: 0.6556
	Classes: 4, 
	Accuracy: 78.9%
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 Performance on test data:
	Loss: 0.5798
	Classes: 4, 
	Accuracy: 81.3%
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 Performance on test data:
	Loss: 0.5427
	Classes: 4, 
	Accuracy: 82.0%
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 Performance on test data:
	Loss: 0.5154
	Classes: 4, 
	Accuracy: 82.2%
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 Performance on test data:
	Loss: 0.4935
	Classes: 4, 
	Accuracy: 82.7%
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 Performance on test data:
	Loss: 0.4752
	Classes: 4, 
	Accuracy: 83.3%
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 Performance on test data:
	Loss: 0.4597
	Classes: 4, 
	Accuracy: 83.8%
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 Performance on test 

In [11]:
import qiskit.tools.jupyter

%qiskit_version_table